In [1]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import time
import math
import community as community_louvain
import networkx.algorithms.community as modularity
from networkx.algorithms.community.quality import modularity
from collections import defaultdict


G = nx.read_edgelist("C:/Users/gnssl/Downloads/facebook_combined.txt", 
                     create_using=nx.Graph(), nodetype=int)

partition_louvain = community_louvain.best_partition(G)
len_partition = len(partition_louvain)

population_size = 200
generations = 1000
crossover_prob = 1.0
mutation_prob = 0.01
local_optimization_frequency=5

min_partition_size = 5
max_partition_size = 15

def initialize_population():
    population = []
    for idx in range(population_size):
        if idx%4==0:
            partition_louvain = community_louvain.best_partition(G)
            partition = [partition_louvain[node] for node in G.nodes]
            population.append(partition)
        else:
            partition = [random.randint(0, max_partition_size) for _ in range(len(G.nodes))]
            population.append(partition)
    return population

def compute_fitness(partition):
    communities = {key: value for key, value in zip(partition_louvain.keys(), partition)}
    new_dict = defaultdict(list)

    for key, value in communities.items():
        new_dict[value].append(key)

    result_dict = dict(new_dict)
    fitness = nx.community.modularity(G, result_dict.values())
    return fitness


def select_parents(population, fitness_values):
    parent1, parent2 = random.sample(population, 2)
    if compute_fitness(parent1) > compute_fitness(parent2):
        return parent1
    else:
        return parent2

def crossover(parent1, parent2):
    size = len(parent1)
    p1, p2 = [0] * size, [0] * size

    # Select two random points in the chromosomes
    point1 = random.randint(0, size - 1)
    point2 = random.randint(0, size - 1)

    # Ensure the two points are not the same
    while point1 == point2:
        point2 = random.randint(0, size - 1)

    # Order the points
    if point1 > point2:
        point1, point2 = point2, point1

    # Copy the respective parts from the parents to the children
    child1 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    child2 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]

    return child1, child2

def mutate(individual, mutation_prob, max_partition_size):
    for i in range(len(individual)):
        if random.random() < mutation_prob:
            # Mutate the community assignment of the node to a random community
            individual[i] = random.randint(0, max_partition_size - 1)
    return individual

def partition_to_communities(partition):
    communities = {}
    for node, community_id in enumerate(partition):
        if community_id in communities:
            communities[community_id].add(node)
        else:
            communities[community_id] = {node}
    return list(communities.values())



def genetic_algorithm():
    population = initialize_population()
    max_fitness_values = []  # Store max fitness values for each generation

    for generation in range(generations):
        fitness_values = [compute_fitness(partition) for partition in population]
        max_fitness = max(fitness_values)
        max_fitness_values.append(max_fitness)
        avg_fitness = sum(fitness_values) / len(fitness_values)
        print('Generation', generation + 1, 'Max:', max_fitness)
        print('Generation', generation + 1, 'Avg:', avg_fitness)

        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parents(population, fitness_values)
            parent2 = select_parents(population, fitness_values)
            if random.random() < crossover_prob:
                child1, child2 = crossover(parent1, parent2)
                
                child1 = mutate(child1, mutation_prob, max_partition_size)
                child2 = mutate(child2, mutation_prob, max_partition_size)
                
                min_fitness_index = min(range(len(fitness_values)), key=fitness_values.__getitem__)
                population[min_fitness_index] = child1
                second_min_fitness_index = min(range(len(fitness_values)), key=lambda x: fitness_values[x] if x != min_fitness_index else float('inf'))
                population[second_min_fitness_index] = child2
                
        
        if generation % local_optimization_frequency == 0:
            random_idx = random.randint(1,population_size//2)
            for idx in range(0, random_idx, population_size//random_idx):
                partition_louvain = community_louvain.best_partition(G)
                partition = [partition_louvain[node] for node in G.nodes]
                population[idx] = partition
       
    # Select the best partition from the final population
    best_partition = max(population, key=compute_fitness)
    return best_partition, max_fitness_values

start = time.time()
best_partition, max_fitness_values = genetic_algorithm()

best_communities = partition_to_communities(best_partition)
end = time.time()
print('time:', end - start)

print("Best Partition:", best_communities)

Generation 1 Max: 0.8358007955742133
Generation 1 Avg: 0.20824407258358463
Generation 2 Max: 0.8358007955742133
Generation 2 Avg: 0.3032968824790658
Generation 3 Max: 0.8358007955742133
Generation 3 Avg: 0.30685172386638626
Generation 4 Max: 0.8358007955742133
Generation 4 Avg: 0.30990556005210074
Generation 5 Max: 0.8358007955742133
Generation 5 Avg: 0.3120142577888718
Generation 6 Max: 0.8358007955742133
Generation 6 Avg: 0.3197127947735352
Generation 7 Max: 0.8358007955742133
Generation 7 Avg: 0.33172749938364254
Generation 8 Max: 0.8358007955742133
Generation 8 Avg: 0.33917536716198293
Generation 9 Max: 0.8358007955742133
Generation 9 Avg: 0.34238252601992514
Generation 10 Max: 0.8358007955742133
Generation 10 Avg: 0.3429205609725169
Generation 11 Max: 0.8358007955742133
Generation 11 Avg: 0.34583123972980423
Generation 12 Max: 0.8358007955742133
Generation 12 Avg: 0.35377036168831394
Generation 13 Max: 0.8358007955742133
Generation 13 Avg: 0.36142180655172185
Generation 14 Max: 0.

Generation 109 Max: 0.8358141477658747
Generation 109 Avg: 0.803518381125934
Generation 110 Max: 0.8358141477658747
Generation 110 Avg: 0.8041356226300942
Generation 111 Max: 0.8358141477658747
Generation 111 Avg: 0.8039986682570921
Generation 112 Max: 0.8358007955742133
Generation 112 Avg: 0.8047993944371591
Generation 113 Max: 0.8358007955742133
Generation 113 Avg: 0.8055457122797586
Generation 114 Max: 0.8358007955742133
Generation 114 Avg: 0.804851261732454
Generation 115 Max: 0.8358007955742133
Generation 115 Avg: 0.8059970543731262
Generation 116 Max: 0.8358007955742133
Generation 116 Avg: 0.805342944129741
Generation 117 Max: 0.8358007955742133
Generation 117 Avg: 0.8066878308625891
Generation 118 Max: 0.8358007955742133
Generation 118 Avg: 0.8073635844939013
Generation 119 Max: 0.8358007955742133
Generation 119 Avg: 0.8074085279681436
Generation 120 Max: 0.8358007955742133
Generation 120 Avg: 0.8076498111684889
Generation 121 Max: 0.8358007955742133
Generation 121 Avg: 0.806878

Generation 215 Max: 0.8358007955742133
Generation 215 Avg: 0.818137276616427
Generation 216 Max: 0.8358007955742133
Generation 216 Avg: 0.8181328076160418
Generation 217 Max: 0.8358007955742133
Generation 217 Avg: 0.8182507420354317
Generation 218 Max: 0.8358007955742133
Generation 218 Avg: 0.8179311097019876
Generation 219 Max: 0.8358007955742133
Generation 219 Avg: 0.8175919605220134
Generation 220 Max: 0.8358007955742133
Generation 220 Avg: 0.8181566155975064
Generation 221 Max: 0.8358007955742133
Generation 221 Avg: 0.8184345867314465
Generation 222 Max: 0.8358045662357076
Generation 222 Avg: 0.8184919753375
Generation 223 Max: 0.8358045662357076
Generation 223 Avg: 0.8183298376035703
Generation 224 Max: 0.8358045662357076
Generation 224 Avg: 0.8183610968822035
Generation 225 Max: 0.8358045662357076
Generation 225 Avg: 0.8169978460439654
Generation 226 Max: 0.8358045662357076
Generation 226 Avg: 0.8174891218416477
Generation 227 Max: 0.8358045662357076
Generation 227 Avg: 0.8184524

Generation 321 Max: 0.8358007955742133
Generation 321 Avg: 0.8217547660108012
Generation 322 Max: 0.8358007955742133
Generation 322 Avg: 0.8219491039806978
Generation 323 Max: 0.8358007955742133
Generation 323 Avg: 0.8219730664747645
Generation 324 Max: 0.8358007955742133
Generation 324 Avg: 0.8218878089632992
Generation 325 Max: 0.8358007955742133
Generation 325 Avg: 0.821301417256753
Generation 326 Max: 0.8358007955742133
Generation 326 Avg: 0.821932821029208
Generation 327 Max: 0.8358007955742133
Generation 327 Avg: 0.8221473763103433
Generation 328 Max: 0.8358007955742133
Generation 328 Avg: 0.8208059834266694
Generation 329 Max: 0.8358007955742133
Generation 329 Avg: 0.8221282786288643
Generation 330 Max: 0.8358007955742133
Generation 330 Avg: 0.8217070631279302
Generation 331 Max: 0.8358007955742133
Generation 331 Avg: 0.822402102637583
Generation 332 Max: 0.8358007955742133
Generation 332 Avg: 0.8217210454800984
Generation 333 Max: 0.8358007955742133
Generation 333 Avg: 0.822034

Generation 427 Max: 0.8358007955742133
Generation 427 Avg: 0.8235598851160654
Generation 428 Max: 0.8358007955742133
Generation 428 Avg: 0.8242294624454675
Generation 429 Max: 0.8358007955742133
Generation 429 Avg: 0.8240885446066722
Generation 430 Max: 0.8358007955742133
Generation 430 Avg: 0.8234514212963978
Generation 431 Max: 0.8358007955742133
Generation 431 Avg: 0.8236572432140321
Generation 432 Max: 0.8358007955742133
Generation 432 Avg: 0.8238884297808683
Generation 433 Max: 0.8358007955742133
Generation 433 Avg: 0.8229506169020566
Generation 434 Max: 0.8358007955742133
Generation 434 Avg: 0.8235089595937499
Generation 435 Max: 0.8358007955742133
Generation 435 Avg: 0.8232657442326874
Generation 436 Max: 0.8358007955742133
Generation 436 Avg: 0.8237261121327388
Generation 437 Max: 0.8358007955742133
Generation 437 Avg: 0.8236958869081343
Generation 438 Max: 0.8358007955742133
Generation 438 Avg: 0.8244168711307664
Generation 439 Max: 0.8358007955742133
Generation 439 Avg: 0.824

Generation 533 Max: 0.8358007955742133
Generation 533 Avg: 0.8248195597076047
Generation 534 Max: 0.8358007955742133
Generation 534 Avg: 0.8243534986828606
Generation 535 Max: 0.8358007955742133
Generation 535 Avg: 0.8248752106961074
Generation 536 Max: 0.8358007955742133
Generation 536 Avg: 0.8251834188995038
Generation 537 Max: 0.8358007955742133
Generation 537 Avg: 0.8252936862245539
Generation 538 Max: 0.8358007955742133
Generation 538 Avg: 0.8248770039070078
Generation 539 Max: 0.8358007955742133
Generation 539 Avg: 0.8240662537270426
Generation 540 Max: 0.8358007955742133
Generation 540 Avg: 0.8246810229874986
Generation 541 Max: 0.8358007955742133
Generation 541 Avg: 0.8237800203509906
Generation 542 Max: 0.8358267952264198
Generation 542 Avg: 0.8250648598419171
Generation 543 Max: 0.8358267952264198
Generation 543 Avg: 0.8252731002075386
Generation 544 Max: 0.8358267952264198
Generation 544 Avg: 0.824634190558664
Generation 545 Max: 0.8358267952264198
Generation 545 Avg: 0.8252

Generation 639 Max: 0.8358007955742133
Generation 639 Avg: 0.8239187053099672
Generation 640 Max: 0.8358007955742133
Generation 640 Avg: 0.8255606989251467
Generation 641 Max: 0.8358007955742133
Generation 641 Avg: 0.8253323802552626
Generation 642 Max: 0.8358007955742133
Generation 642 Avg: 0.824862226889744
Generation 643 Max: 0.8358007955742133
Generation 643 Avg: 0.8255905780653108
Generation 644 Max: 0.8358007955742133
Generation 644 Avg: 0.8247540760280441
Generation 645 Max: 0.8358007955742133
Generation 645 Avg: 0.8252908224694143
Generation 646 Max: 0.8358007955742133
Generation 646 Avg: 0.8250277413672552
Generation 647 Max: 0.8358007955742133
Generation 647 Avg: 0.8252033071765177
Generation 648 Max: 0.8358007955742133
Generation 648 Avg: 0.8249237813291798
Generation 649 Max: 0.8358007955742133
Generation 649 Avg: 0.8253909440288719
Generation 650 Max: 0.8358007955742133
Generation 650 Avg: 0.8254389844607997
Generation 651 Max: 0.8358007955742133
Generation 651 Avg: 0.8251

Generation 745 Max: 0.8358412575959795
Generation 745 Avg: 0.8258820194328926
Generation 746 Max: 0.8358412575959795
Generation 746 Avg: 0.8260827856542619
Generation 747 Max: 0.8358412575959795
Generation 747 Avg: 0.826057879513798
Generation 748 Max: 0.8358412575959795
Generation 748 Avg: 0.824830388105762
Generation 749 Max: 0.8358412575959795
Generation 749 Avg: 0.8252938614092902
Generation 750 Max: 0.8358412575959795
Generation 750 Avg: 0.825305181475413
Generation 751 Max: 0.8358412575959795
Generation 751 Avg: 0.8256378329712333
Generation 752 Max: 0.8358412575959795
Generation 752 Avg: 0.8258086561873452
Generation 753 Max: 0.8358412575959795
Generation 753 Avg: 0.8249388831450887
Generation 754 Max: 0.8358412575959795
Generation 754 Avg: 0.8262025378974773
Generation 755 Max: 0.8358412575959795
Generation 755 Avg: 0.8257897131164104
Generation 756 Max: 0.8358412575959795
Generation 756 Avg: 0.8259365986678161
Generation 757 Max: 0.8358412575959795
Generation 757 Avg: 0.824835

Generation 851 Max: 0.8358007955742133
Generation 851 Avg: 0.8258967556668857
Generation 852 Max: 0.8358007955742133
Generation 852 Avg: 0.8265206315587786
Generation 853 Max: 0.8358007955742133
Generation 853 Avg: 0.8251821799795681
Generation 854 Max: 0.8358007955742133
Generation 854 Avg: 0.8249541474486286
Generation 855 Max: 0.8358007955742133
Generation 855 Avg: 0.8250465898598449
Generation 856 Max: 0.8358007955742133
Generation 856 Avg: 0.8259641325106074
Generation 857 Max: 0.8358007955742133
Generation 857 Avg: 0.8258032863147814
Generation 858 Max: 0.8358007955742133
Generation 858 Avg: 0.8264357231249185
Generation 859 Max: 0.8358007955742133
Generation 859 Avg: 0.8261972251817992
Generation 860 Max: 0.8358007955742133
Generation 860 Avg: 0.826274077348605
Generation 861 Max: 0.8358007955742133
Generation 861 Avg: 0.8266057820494416
Generation 862 Max: 0.8358007955742133
Generation 862 Avg: 0.8256175836966199
Generation 863 Max: 0.8358007955742133
Generation 863 Avg: 0.8255

Generation 957 Max: 0.8358007955742133
Generation 957 Avg: 0.8262866133125693
Generation 958 Max: 0.8358007955742133
Generation 958 Avg: 0.8269633865638635
Generation 959 Max: 0.8358007955742133
Generation 959 Avg: 0.8260058382501712
Generation 960 Max: 0.8358007955742133
Generation 960 Avg: 0.8261617407469397
Generation 961 Max: 0.8358007955742133
Generation 961 Avg: 0.8257582181298487
Generation 962 Max: 0.8358007955742133
Generation 962 Avg: 0.825189294298311
Generation 963 Max: 0.8358007955742133
Generation 963 Avg: 0.8266957663041903
Generation 964 Max: 0.8358007955742133
Generation 964 Avg: 0.8266871429186905
Generation 965 Max: 0.8358007955742133
Generation 965 Avg: 0.8269256932282195
Generation 966 Max: 0.8358007955742133
Generation 966 Avg: 0.8269101645728001
Generation 967 Max: 0.8358007955742133
Generation 967 Avg: 0.8268930634851941
Generation 968 Max: 0.8358007955742133
Generation 968 Avg: 0.826001760264769
Generation 969 Max: 0.8358007955742133
Generation 969 Avg: 0.82658